Jupyter Notebook for Selecting Lipid MRMs and OzESIs after a Neutral Loss Scan

In [133]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from IPython.display import Image

First perform a neutral loss scan and obtain pecursor ion m/z values and the neutral loss. Upload an excel spreadsheet with these values listed in two columns.

![title](Figures/search_list.png)

A database of known lipid MRMs will be parsed and matched. And the rule-based method for OzESI will be used. 


![title](Figures/OzESI_results.png)

Load MRM databases 1: SUPPLE_2.XLS and 2:ListMRMs.csv, and then save as a pandas dataframe. 

In [134]:
# # #loop through all sheets in SUPPLE_2.XLS and make a df of Compound Name, Parent Ion, and Product Ion
# mrm_list_new = pd.read_excel('lipid_database/Lipid_Database.xlsx', sheet_name = None)
# mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
# mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion']]
# #mrm_list_official = mrm_list_new.loc[:, ['Compound Name', 'Parent Ion', 'Product Ion']]
# #Add underscore to middle of columns names
# mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
# #round the Parent Ion and Product Ion to 1 decimal place
# mrm_list_offical.loc[:, 'Parent_Ion'] = mrm_list_offical['Parent_Ion'].round(0)
# mrm_list_offical.loc[:, 'Product_Ion'] = mrm_list_offical['Product_Ion'].round(0)
# # Create transition column by combining Parent Ion and Product Ion with arrow between numbers
# mrm_list_offical.loc[:, 'Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
# # Change column compound name to lipid
# mrm_database = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})




# # print(mrm_list_offical.head(25))
# # print(df_mrm.head(25))

def read_mrm_list(filename):
    mrm_list_new = pd.read_excel(filename, sheet_name=None)
    mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
    mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion', 'Class']]
    # Add underscore to middle of columns names
    mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
    # Round Parent Ion and Product Ion to 1 decimal place
    mrm_list_offical['Parent_Ion'] = np.floor(mrm_list_offical['Parent_Ion'].round(0))
    mrm_list_offical['Product_Ion'] = np.floor(mrm_list_offical['Product_Ion'].round(0))
    # Create transition column by combining Parent Ion and Product Ion with arrow between numbers
    mrm_list_offical['Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
    # Change column compound name to lipid
    mrm_list_offical = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
    # Make a column called Class match lipid column to lipid types
    return mrm_list_offical

mrm_database = read_mrm_list('lipid_database/Lipid_Database.xlsx')
mrm_database.tail()

/tmp/ipykernel_662/781428457.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Parent_Ion'] = np.floor(mrm_list_offical['Parent_Ion'].round(0))
/tmp/ipykernel_662/781428457.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Product_Ion'] = np.floor(mrm_list_offical['Product_Ion'].round(0))
/tmp/ipykernel_662/781428457.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,Lipid,Parent_Ion,Product_Ion,Class,Transition
3264,STD_15:0-18:1(d7) PI (NH4 Salt),848.0,570.0,STD_15:0-18:1(d7) PI (NH4 Salt),848.0 -> 570.0
3265,STD_15:0-18:1(d7) PS (Na Salt),756.0,570.0,STD_15:0-18:1(d7) PS (Na Salt),756.0 -> 570.0
3266,STD_15:0-18:1(d7)-15:0 TAG,830.0,570.0,STD_15:0-18:1(d7)-15:0 TAG,830.0 -> 570.0
3267,STD_18:1(d7) Chol Ester,676.0,369.0,STD_18:1(d7) Chol Ester,676.0 -> 369.0
3268,STD_d18:1-18:1(d9) SM,739.0,184.0,STD_d18:1-18:1(d9) SM,739.0 -> 184.0


Input search list excel sheet as discussed in the introduction. Save the data into a dataframe and then search the databases for matches and output them into a found_list dataframe. 

In [143]:
####### Two columns Parent and Neutral Loss

# search_list = pd.read_excel('./data_OzESI/search/LiverLDs_NL_042523.xlsx')
# print(search_list.head(10))
# found_list = pd.DataFrame(columns=['Lipid','Parent_Ion','Product_Ion','Class','Transition'])
# # print(search_list.head(10))
# search_list['Product_Ion'] = search_list['Parent_Ion'] - search_list['neutral_loss']
# #Round search list columns to 0 deciaml places
# search_list['Parent_Ion'] = search_list['Parent_Ion'].round(0)
# search_list['Product_Ion'] = search_list['Product_Ion'].round(0)

# print(search_list.tail())

# #Search for matches between mrm_list_offical and search_list for Parent Ion and Product Ion
# for i in range(len(mrm_database)):
#     for j in range(len(search_list)):
#         if mrm_database.loc[i,'Parent_Ion'] == search_list.loc[j,'Parent_Ion'] and mrm_database.loc[i,'Product_Ion'] == search_list.loc[j,'Product_Ion']:
#             found_list = found_list.append(mrm_database.loc[i,:])



# # found_list.to_excel('./data_OzESI/data_csv/found_list.xlsx', index=False)      
# found_list.head(25)


In [171]:
search_list = pd.read_excel('./data_OzESI/search/LiverLDs_NL_042523.xlsx')

found_list = pd.DataFrame(columns=['Lipid','Parent_Ion','Product_Ion','Neutral_Loss','Class','Transition'])
#Round search list columns to 0 deciaml places
search_list['Parent_Ion'] = search_list['Parent_Ion'].round(0)
search_list['Product_Ion'] = search_list['Product_Ion'].round(0)
search_list['Neutral_Loss'] = search_list['Parent_Ion'] - search_list['Product_Ion']
print(search_list.head(10))
#Search for matches between mrm_list_offical and search_list for Parent Ion and Product Ion
for i in range(len(mrm_database)):
    for j in range(len(search_list)):
        if mrm_database.loc[i,'Parent_Ion'] == search_list.loc[j,'Parent_Ion'] and mrm_database.loc[i,'Product_Ion'] == search_list.loc[j,'Product_Ion']:
            found_list = found_list.append(mrm_database.loc[i,:])
            #add neutral loss to found_list
            found_list.loc[i,'Neutral_Loss'] = search_list.loc[j,'Neutral_Loss']

# # Set multi-index on the search_list for faster lookups
# search_list_idx = search_list.set_index(['Parent_Ion', 'Product_Ion'])

# # Filter the mrm_database by matches in the search_list
# found_list = mrm_database[
#     mrm_database[['Parent_Ion', 'Product_Ion']]
#     .apply(lambda x: tuple(x), axis=1)
#     .isin(search_list_idx.index)
# ]

# search_list_idx = search_list_idx.reset_index()
# # Add the neutral loss from the search_list to the found_list
# found_list['Neutral_Loss'] = search_list_idx['Neutral_Loss'].reindex(found_list.index).values



found_list.head(25)




   Parent_Ion              Lipid  Product_Ion  Neutral_Loss
0       847.0   TG(50:3)_FA 18:1        548.0         299.0
1       863.0   TG(52:9)_FA 18:1        563.0         300.0
2       887.0  TG(54:11)_FA 18:1        588.0         299.0
3       897.0   TG(54:6)_FA 18:1        597.0         300.0
4       899.0   TG(54:5)_FA 18:1        599.0         300.0
5       901.0   TG(54:4)_FA 18:1        602.0         299.0
6       905.0   TG(54:2)_FA 18:1        606.0         299.0
7       923.0   TG(56:7)_FA 18:1        623.0         300.0
8       931.0   TG(56:3)_FA 18:1        632.0         299.0
9       949.0   TG(58:8)_FA 18:1        650.0         299.0


/tmp/ipykernel_662/1133954860.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_database.loc[i,:])
/tmp/ipykernel_662/1133954860.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_database.loc[i,:])
/tmp/ipykernel_662/1133954860.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_database.loc[i,:])
/tmp/ipykernel_662/1133954860.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_database.loc[i,:])
/tmp/ipykernel_662/1133954860.py:13: FutureWarning: The frame.append method is deprecated and wi

,Lipid,Parent_Ion,Product_Ion,Neutral_Loss,Class,Transition
590,"[TG(44:0),TG(O-45:0)]_FA14:0",769.0,524.0,245.0,TAG,769.0 -> 524.0
602,[TG(46:0)]_FA14:0,797.0,552.0,245.0,TAG,797.0 -> 552.0
603,[TG(46:1)]_FA14:0,795.0,550.0,245.0,TAG,795.0 -> 550.0
615,[TG(48:3)]_FA14:0,819.0,574.0,245.0,TAG,819.0 -> 574.0
625,"[TG(49:7),TG(48:0)]_FA14:0",825.0,580.0,245.0,TAG,825.0 -> 580.0
626,"[TG(49:8),TG(48:1)]_FA14:0",823.0,578.0,245.0,TAG,823.0 -> 578.0
627,[TG(50:3)]_FA14:0,847.0,602.0,245.0,TAG,847.0 -> 602.0
628,[TG(50:4)]_FA14:0,845.0,600.0,245.0,TAG,845.0 -> 600.0
637,"[TG(51:8),TG(50:1)]_FA14:0",851.0,606.0,245.0,TAG,851.0 -> 606.0
638,"[TG(51:9),TG(50:2)]_FA14:0",849.0,604.0,245.0,TAG,849.0 -> 604.0


Create a dataframe of DB_position and aldehyde ion based off lipid rules. Save in dataframe df_OzESI

In [172]:
df_OzESI = pd.DataFrame(columns=['DB_Position','Aldehyde_Ion'])

for i in range(3,21):
    df_OzESI.loc[i,'DB_Position'] = i
    df_OzESI.loc[i,'Aldehyde_Ion'] = 26 + (14 * (i-3))

df_OzESI.head(25)


,DB_Position,Aldehyde_Ion
3,3,26
4,4,40
5,5,54
6,6,68
7,7,82
8,8,96
9,9,110
10,10,124
11,11,138
12,12,152


Input the requested OzESI n# in the OzESI_list. Based off the previously found data in the found_list, the script will provide the correct m/z value with each n# for each lipid in the found list dataframe

In [174]:
OzESI_list = [3,5,7,9,11]
starting_column = 6
last_column = 11
#create columns in found list called n-# based on the numbers in OzESI_list
for i in OzESI_list:
    found_list['n-' + str(i)] = ''
print(found_list.head(25))

# Parent_Ion - Aldehyde_Ion *

# Subtract the aldehyde ion from the parent ion with the corresponding DB position and add the result to the found_list dataframe
for i in range(3,len(df_OzESI)):
    for j in range(len(found_list)):
        #if df_OzESI.loc[i,'DB_Position']:
        for k in range(starting_column,last_column):
            if df_OzESI.loc[i,'DB_Position'] == OzESI_list[k-starting_column]:
                found_list.iloc[j,k] = found_list.iloc[j,1] - df_OzESI.loc[i,'Aldehyde_Ion']

# for i, row in df_OzESI.iloc[3:].iterrows():
#     mask = df_OzESI['DB_Position'] == row['DB_Position']
#     found_list.loc[mask, OzESI_list] = found_list.loc[mask, ['Parent_Ion']].sub(row['Aldehyde_Ion'] - 1)

        
found_list.head(25)

                            Lipid  Parent_Ion  Product_Ion Neutral_Loss Class  \
590  [TG(44:0),TG(O-45:0)]_FA14:0       769.0        524.0        245.0   TAG   
602             [TG(46:0)]_FA14:0       797.0        552.0        245.0   TAG   
603             [TG(46:1)]_FA14:0       795.0        550.0        245.0   TAG   
615             [TG(48:3)]_FA14:0       819.0        574.0        245.0   TAG   
625    [TG(49:7),TG(48:0)]_FA14:0       825.0        580.0        245.0   TAG   
626    [TG(49:8),TG(48:1)]_FA14:0       823.0        578.0        245.0   TAG   
627             [TG(50:3)]_FA14:0       847.0        602.0        245.0   TAG   
628             [TG(50:4)]_FA14:0       845.0        600.0        245.0   TAG   
637    [TG(51:8),TG(50:1)]_FA14:0       851.0        606.0        245.0   TAG   
638    [TG(51:9),TG(50:2)]_FA14:0       849.0        604.0        245.0   TAG   
640             [TG(52:4)]_FA14:0       873.0        628.0        245.0   TAG   
643    [TG(52:8),TG(51:1)]_F

,Lipid,Parent_Ion,Product_Ion,Neutral_Loss,Class,Transition,n-3,n-5,n-7,n-9,n-11
590,"[TG(44:0),TG(O-45:0)]_FA14:0",769.0,524.0,245.0,TAG,769.0 -> 524.0,743.0,715.0,687.0,659.0,631.0
602,[TG(46:0)]_FA14:0,797.0,552.0,245.0,TAG,797.0 -> 552.0,771.0,743.0,715.0,687.0,659.0
603,[TG(46:1)]_FA14:0,795.0,550.0,245.0,TAG,795.0 -> 550.0,769.0,741.0,713.0,685.0,657.0
615,[TG(48:3)]_FA14:0,819.0,574.0,245.0,TAG,819.0 -> 574.0,793.0,765.0,737.0,709.0,681.0
625,"[TG(49:7),TG(48:0)]_FA14:0",825.0,580.0,245.0,TAG,825.0 -> 580.0,799.0,771.0,743.0,715.0,687.0
626,"[TG(49:8),TG(48:1)]_FA14:0",823.0,578.0,245.0,TAG,823.0 -> 578.0,797.0,769.0,741.0,713.0,685.0
627,[TG(50:3)]_FA14:0,847.0,602.0,245.0,TAG,847.0 -> 602.0,821.0,793.0,765.0,737.0,709.0
628,[TG(50:4)]_FA14:0,845.0,600.0,245.0,TAG,845.0 -> 600.0,819.0,791.0,763.0,735.0,707.0
637,"[TG(51:8),TG(50:1)]_FA14:0",851.0,606.0,245.0,TAG,851.0 -> 606.0,825.0,797.0,769.0,741.0,713.0
638,"[TG(51:9),TG(50:2)]_FA14:0",849.0,604.0,245.0,TAG,849.0 -> 604.0,823.0,795.0,767.0,739.0,711.0


In [128]:
#print datatype of Parent_Ion column and n-7 column
print(found_list['Parent_Ion'].dtypes)
print(found_list['n-7'].dtypes)

#change datatype of all n-# columns to float64
for i in range(5,8):
    found_list.iloc[:,i] = found_list.iloc[:,i].astype('float64')

print(found_list['Parent_Ion'].dtypes)
print(found_list['n-7'].dtypes)
print(found_list['Product_Ion'].dtypes)



float64
object
float64
float64
float64


/tmp/ipykernel_662/3566798862.py:7: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  found_list.iloc[:,i] = found_list.iloc[:,i].astype('float64')


Save the dataframe as an excel sheet that can be downloaded and given back to the mass spec chemist

In [175]:
from datetime import date
name_of_sample = 'LiverLDs_NL_042523'
i = 0
while i < 100:
    if not os.path.exists('./data_OzESI/data_excel/OzESI_Selections_{}_.xlsx'.format(name_of_sample)):
        found_list.to_excel('./data_OzESI/data_excel/OzESI_Selections_{}_.xlsx'.format(name_of_sample), index=False)
        break
    elif not os.path.exists('./data_OzESI/data_excel/OzESI_Selections_{}_'.format(name_of_sample) + str(i) + '.xlsx'):
        found_list.to_excel('./data_OzESI/data_excel/OzESI_Selections_{}_'.format(name_of_sample) + str(i) + '.xlsx', index=False)
        break
    else:
        i += 1


In [176]:
# matching_list = pd.DataFrame(columns=['Lipid','Parent_Ion','Product_Ion','Class','Transition'])

# #if n-7 and product ion are the same for different lipids, add them to the matching list
# for i in range(len(found_list)):
#     for j in range(len(found_list)):
#         if found_list.iloc[i,5] == found_list.iloc[j,2] and i != j:
#             matching_list = matching_list.append(found_list.iloc[i,:])

# matching_list.head(25)

,Lipid,Parent_Ion,Product_Ion,Class,Transition


In [178]:
# matching_lipids_df = pd.DataFrame(columns=['Matching_Lipids','Parent_Ion','n-7'])
# matching_lipids = {}
# # Group the DataFrame by 'Product_Ion' and 'n-3'
# grouped = found_list.groupby(['Parent_Ion', 'n-3'])

# # Iterate over the groups
# for group_name, group_df in grouped:
#     # Get the list of lipids in the group
#     lipids = list(group_df['Lipid'])
    
#     # Check if there are multiple lipids in the group
#     if len(lipids) > 0:
#         # Add the lipids to the matching_lipids dictionary
#         matching_lipids[group_name] = lipids
#         matching_lipids_df = matching_lipids_df.append({
#             'Matching_Lipids': lipids,                              
#             'Parent_Ion': group_name[0],
#             'n-3': group_name[1]
            
#         }, ignore_index=True)

    

# # Group the DataFrame by 'Product_Ion' and 'n-5'
# grouped = found_list.groupby(['Parent_Ion', 'n-5'])

# # Iterate over the groups
# for group_name, group_df in grouped:
#     # Get the list of lipids in the group
#     lipids = list(group_df['Lipid'])

#     # Check if there are multiple lipids in the group
#     if len(lipids) > 0:
#         # Add the lipids to the matching_lipids dictionary
#         matching_lipids[group_name] = lipids
#         matching_lipids_df = matching_lipids_df.append({
#             'Matching_Lipids': lipids,                              
#             'Parent_Ion': group_name[0],
#             'n-5': group_name[1]
            
#         }, ignore_index=True)

# # Group the DataFrame by 'Product_Ion' and 'n-7'


# grouped = found_list.groupby(['Parent_Ion', 'n-7'])
# print(grouped.tail())

# # Create an empty dictionary to store the matching lipids


# # Iterate over the groups
# for group_name, group_df in grouped:
#     # Get the list of lipids in the group
#     lipids = list(group_df['Lipid'])
    
#     # Check if there are multiple lipids in the group
#     if len(lipids) > 0:
#         # Add the lipids to the matching_lipids dictionary
#         matching_lipids[group_name] = lipids
#         matching_lipids_df = matching_lipids_df.append({
#             'Matching_Lipids': lipids,                              
#             'Parent_Ion': group_name[0],
#             'n-7': group_name[1]
            
#         }, ignore_index=True)

# # Group the DataFrame by 'Product_Ion' and 'n-7'
# grouped = found_list.groupby(['Parent_Ion', 'n-9'])

# # Create an empty dictionary to store the matching lipids
# matching_lipids = {}

# # Iterate over the groups
# for group_name, group_df in grouped:
#     # Get the list of lipids in the group
#     lipids = list(group_df['Lipid'])
    
#     # Check if there are multiple lipids in the group
#     if len(lipids) > 0:
#         # Add the lipids to the matching_lipids dictionary
#         matching_lipids[group_name] = lipids
#         matching_lipids_df = matching_lipids_df.append({
#             'Matching_Lipids': lipids,
#             'Parent_Ion': group_name[0],
#             'n-9': group_name[1]
            
#         }, ignore_index=True)

# #remove extra brackets from matching_lipids_df
# matching_lipids_df['Matching_Lipids'] = matching_lipids_df['Matching_Lipids'].str[0]
# # Print the matching lipids
# print(matching_lipids)
# matching_lipids_df.tail(25)


In [109]:
# # Group the DataFrame by 'Product_Ion' and 'n-7'
# grouped = found_list.groupby(['Product_Ion', 'n-11'])

# # Create an empty dictionary to store the matching lipids
# matching_lipids = {}

# # Iterate over the groups
# for group_name, group_df in grouped:
#     # Get the list of lipids in the group
#     lipids = list(group_df['Lipid'])
    
#     # Check if there are multiple lipids in the group
#     if len(lipids) > 0:
#         # Add the lipids to the matching_lipids dictionary
#         matching_lipids[group_name] = lipids
#         matching_lipids_df = matching_lipids_df.append({
#             'Matching_Lipids': lipids,
#             'Product_Ion': group_name[0],
#             'n-11': group_name[1]
            
#         }, ignore_index=True)


# # Print the matching lipids
# matching_lipids_df.tail(25)

In [105]:
# # Group the DataFrame by 'Product_Ion' and 'n-7'
# grouped = found_list.groupby(['Product_Ion', 'n-7'])

# # Create an empty DataFrame to store the matching lipids
# matching_lipids_df = pd.DataFrame(columns=['Product_Ion', 'n-7', 'Matching_Lipids'])

# # Iterate over the groups
# for group_name, group_df in grouped:
#     # Get the list of lipids in the group
#     lipids = list(group_df['Lipid'])
    
#     # Check if there are multiple lipids in the group
#     if len(lipids) > 0:
#         # Add the group to the matching_lipids_df DataFrame
#         matching_lipids_df = matching_lipids_df.append({
#             'Product_Ion': group_name[0],
#             'n-7': group_name[1],
#             'Matching_Lipids': lipids
#         }, ignore_index=True)

# # Print the matching lipids DataFrame
# print(matching_lipids_df.head(25))
# matching_lipids_df.to_excel('./data_OzESI/data_excel/FA 18:1.xlsx', index=False)


In [106]:
# # Group the DataFrame by 'Product_Ion' and 'n-7'
# grouped = found_list.groupby(['Product_Ion', 'n-11'])

# # Create an empty DataFrame to store the matching lipids
# matching_lipids_df = pd.DataFrame(columns=['Product_Ion', 'n-11', 'Matching_Lipids'])

# # Iterate over the groups
# for group_name, group_df in grouped:
#     # Get the list of lipids in the group
#     lipids = list(group_df['Lipid'])
    
#     # Check if there are multiple lipids in the group
#     if len(lipids) > 0:
#         # Add the group to the matching_lipids_df DataFrame
#         matching_lipids_df = matching_lipids_df.append({
#             'Product_Ion': group_name[0],
#             'n-11': group_name[1],
#             'Matching_Lipids': lipids
#         }, ignore_index=True)

# # Print the matching lipids DataFrame
# matching_lipids_df.head(25)
# matching_lipids_df.to_excel('./data_OzESI/data_excel/FA 20:1.xlsx', index=False)
